In [1]:
!pip install keras

In [5]:
keras.__version__

'2.4.3'

In [95]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import keras
import glob
import pickle

from music21 import converter, instrument, stream, note, chord
import tensorflow as tf
import music21 as m21


from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation, Bidirectional, Flatten
from keras import utils
from keras.callbacks import ModelCheckpoint
from keras_self_attention import SeqSelfAttention
from keras.layers import BatchNormalization as BatchNorm
from tensorflow.keras.layers import TimeDistributed

In [145]:
pickle_file = open("data/notes", "rb")
notes = pickle.load(pickle_file)


In [137]:
#using these just to test
VOCAB = (len(set(notes)))
pitches = sorted(set(note for note in notes))
lstm_input, lstm_normalized = prepare_notes(notes, pitches, VOCAB)

In [83]:
def build_model(lstm_input):
    '''
    Build and compile the model
    
    lstm_input: lstm_input.shape[1] = number of steps, lstm_input.shape[2] = number features needed for
    Bi-directional
    
    model_output: number of categories to output for classification
    
    returns: compiled model
    '''
    
    model = Sequential()
    model.add(Bidirectional(LSTM(512,input_shape=(lstm_input.shape[1], lstm_input.shape[2]), return_sequences=True)))
    model.add(SeqSelfAttention(attention_activation='sigmoid'))
    model.add(TimeDistributed(BatchNorm()))
    model.add(Dropout(0.3))
    model.add(BatchNorm())
    
    model.add(Bidirectional(LSTM(512,input_shape=(lstm_input.shape[1], lstm_input.shape[2]), return_sequences=True)))
    model.add(SeqSelfAttention(attention_activation='sigmoid'))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(LSTM(512,input_shape=(lstm_input.shape[1], lstm_input.shape[2]), return_sequences=False)))
    model.add(SeqSelfAttention(attention_activation='sigmoid'))
    model.add(Dropout(0.3))
    model.add(BatchNorm())
    
    model.add(Flatten())
    model.add(Dense(VOCAB))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    
    #model.load_weights('models_weights/weights_attention.h5')
    
    
    return model

In [138]:
def prepare_notes(notes, pitches, VOCAB):
    '''
    prepare lstm input notes again used by the network to predict notes
    
    '''
    #setting the sequence length to 100
    #print(len(set(notes)))
    sequence = 100 
    
    #creating the note to int dict to map pitches to integers
    note_dict = dict((note, number) for number, note in enumerate(pitches))
    #print(note_dict)
    lstm_input = []
    lstm_output = []
    
    #creating inputs and corresponding outputs
    for i in range(0, len(notes)- sequence, 1):
        inputs = notes[i : i + sequence]
        outputs = notes[i + sequence]
        lstm_input.append([note_dict[pitch] for pitch in inputs])
        lstm_output.append(note_dict[outputs])
    
    #creating all the objects to reshape network input to make compatable with lstm network
    shape_1 = lstm_input
    shape_2 = len(lstm_input)
    shape_3 = sequence 
    
    #reshaping lstm input for lstm
    lstm_input = np.reshape(shape_1, (shape_2, shape_3, 1))
    
    #normalize lstm input with  number of unique notes
    lstm_normalized = lstm_input / float(len(pitches))
    
    return (lstm_input, lstm_normalized)

In [142]:
def predict_to_notes(model, lstm_input, pitches, VOCAB):
    '''
    **Generated predictions from model based on random starting point**
    
    model: --> Original model with weights loaded
    lstm_inputL: --> Output from prepare, used to initialize pattern with an int the model recognizes
    pitches: ---> list of all the notes, chords and rests
    VOCAB:----> Number of unique notes to classify = len(pitches)
    
    output: Predicted_notes 
    '''
    #random starting point 
    start = np.random.randint(0, len(lstm_input) -1)
    note_dict = dict((number, note) for number, note in enumerate(pitches))
    
    pattern = lstm_input[start]
    print(type(pattern))
   
    predicted_notes = []
    
    
    for note in range(500):
        to_predict = np.reshape(pattern, (1, len(pattern), 1))
        to_predict = to_predict/ float(VOCAB)
        #print(to_predict.shape)
        
        prediction = model.predict(to_predict, verbose = 0)
        #print(type(prediction))
        index = np.argmax(prediction)
        print(index)
        
        result = note_dict[index]
        predicted_notes.append(result)
        
        pattern = np.append(pattern, index)
        print(pattern)
        pattern = pattern[1:len(pattern)]
        
    return predicted_notes
predicted_notes = predict_to_notes(model, lstm_input, pitches, VOCAB)

<class 'numpy.ndarray'>
2045
[ 849 1317 1833  474 2064 1111 1126 1327  321 2103 1891 1744 1237 1580
 1911 1317  486 2259 1771 2205 2258 2185 1344  490 1815 1771 1357 1563
 1791 1885 1232 2100 1801 2088 1238 1369 2370 2063 2109 2205 2342 1212
 1560 2336 2063 1212 1560 2063 2107 1212 1560 2336 2063 1212 1560 2063
 1061 2107 1212 1560 2336 2063 1212 1560 2063  280 2107 1212 1560 2336
 2063 1212 1560 2063  403 2107 1212 1560 2336 2063 1212 1560 2063 2107
 1212 1560 2336 2063 1212 1560 2063  122 2107 1212 1560 2336 2063 1212
 1560 2063 2045]
1703
[1317 1833  474 2064 1111 1126 1327  321 2103 1891 1744 1237 1580 1911
 1317  486 2259 1771 2205 2258 2185 1344  490 1815 1771 1357 1563 1791
 1885 1232 2100 1801 2088 1238 1369 2370 2063 2109 2205 2342 1212 1560
 2336 2063 1212 1560 2063 2107 1212 1560 2336 2063 1212 1560 2063 1061
 2107 1212 1560 2336 2063 1212 1560 2063  280 2107 1212 1560 2336 2063
 1212 1560 2063  403 2107 1212 1560 2336 2063 1212 1560 2063 2107 1212
 1560 2336 2063 1212 1560 

2052
[ 486 2259 1771 2205 2258 2185 1344  490 1815 1771 1357 1563 1791 1885
 1232 2100 1801 2088 1238 1369 2370 2063 2109 2205 2342 1212 1560 2336
 2063 1212 1560 2063 2107 1212 1560 2336 2063 1212 1560 2063 1061 2107
 1212 1560 2336 2063 1212 1560 2063  280 2107 1212 1560 2336 2063 1212
 1560 2063  403 2107 1212 1560 2336 2063 1212 1560 2063 2107 1212 1560
 2336 2063 1212 1560 2063  122 2107 1212 1560 2336 2063 1212 1560 2063
 2045 1703 2045 1412 1508 2051 2053 1412 1153 2051 2053 2053   69  958
 2053  803 2052]
1954
[2259 1771 2205 2258 2185 1344  490 1815 1771 1357 1563 1791 1885 1232
 2100 1801 2088 1238 1369 2370 2063 2109 2205 2342 1212 1560 2336 2063
 1212 1560 2063 2107 1212 1560 2336 2063 1212 1560 2063 1061 2107 1212
 1560 2336 2063 1212 1560 2063  280 2107 1212 1560 2336 2063 1212 1560
 2063  403 2107 1212 1560 2336 2063 1212 1560 2063 2107 1212 1560 2336
 2063 1212 1560 2063  122 2107 1212 1560 2336 2063 1212 1560 2063 2045
 1703 2045 1412 1508 2051 2053 1412 1153 2051 2053

2044
[1801 2088 1238 1369 2370 2063 2109 2205 2342 1212 1560 2336 2063 1212
 1560 2063 2107 1212 1560 2336 2063 1212 1560 2063 1061 2107 1212 1560
 2336 2063 1212 1560 2063  280 2107 1212 1560 2336 2063 1212 1560 2063
  403 2107 1212 1560 2336 2063 1212 1560 2063 2107 1212 1560 2336 2063
 1212 1560 2063  122 2107 1212 1560 2336 2063 1212 1560 2063 2045 1703
 2045 1412 1508 2051 2053 1412 1153 2051 2053 2053   69  958 2053  803
 2052 1954  979 1418 2051  524  527 2053  649 2053  280 2053  524  521
 2010  649 2044]
6
[2088 1238 1369 2370 2063 2109 2205 2342 1212 1560 2336 2063 1212 1560
 2063 2107 1212 1560 2336 2063 1212 1560 2063 1061 2107 1212 1560 2336
 2063 1212 1560 2063  280 2107 1212 1560 2336 2063 1212 1560 2063  403
 2107 1212 1560 2336 2063 1212 1560 2063 2107 1212 1560 2336 2063 1212
 1560 2063  122 2107 1212 1560 2336 2063 1212 1560 2063 2045 1703 2045
 1412 1508 2051 2053 1412 1153 2051 2053 2053   69  958 2053  803 2052
 1954  979 1418 2051  524  527 2053  649 2053  280 20

2308
[2107 1212 1560 2336 2063 1212 1560 2063 1061 2107 1212 1560 2336 2063
 1212 1560 2063  280 2107 1212 1560 2336 2063 1212 1560 2063  403 2107
 1212 1560 2336 2063 1212 1560 2063 2107 1212 1560 2336 2063 1212 1560
 2063  122 2107 1212 1560 2336 2063 1212 1560 2063 2045 1703 2045 1412
 1508 2051 2053 1412 1153 2051 2053 2053   69  958 2053  803 2052 1954
  979 1418 2051  524  527 2053  649 2053  280 2053  524  521 2010  649
 2044    6 2333 1446    6 1432 2044 2333  569  643 1171 2153 2202  527
  649 2046 2308]
1103
[1212 1560 2336 2063 1212 1560 2063 1061 2107 1212 1560 2336 2063 1212
 1560 2063  280 2107 1212 1560 2336 2063 1212 1560 2063  403 2107 1212
 1560 2336 2063 1212 1560 2063 2107 1212 1560 2336 2063 1212 1560 2063
  122 2107 1212 1560 2336 2063 1212 1560 2063 2045 1703 2045 1412 1508
 2051 2053 1412 1153 2051 2053 2053   69  958 2053  803 2052 1954  979
 1418 2051  524  527 2053  649 2053  280 2053  524  521 2010  649 2044
    6 2333 1446    6 1432 2044 2333  569  643 1171

571
[2063  280 2107 1212 1560 2336 2063 1212 1560 2063  403 2107 1212 1560
 2336 2063 1212 1560 2063 2107 1212 1560 2336 2063 1212 1560 2063  122
 2107 1212 1560 2336 2063 1212 1560 2063 2045 1703 2045 1412 1508 2051
 2053 1412 1153 2051 2053 2053   69  958 2053  803 2052 1954  979 1418
 2051  524  527 2053  649 2053  280 2053  524  521 2010  649 2044    6
 2333 1446    6 1432 2044 2333  569  643 1171 2153 2202  527  649 2046
 2308 1103 1335  816 1789 2046  527 1045 1077 2046  612 1191 1990 1777
  869 1541  571]
1509
[ 280 2107 1212 1560 2336 2063 1212 1560 2063  403 2107 1212 1560 2336
 2063 1212 1560 2063 2107 1212 1560 2336 2063 1212 1560 2063  122 2107
 1212 1560 2336 2063 1212 1560 2063 2045 1703 2045 1412 1508 2051 2053
 1412 1153 2051 2053 2053   69  958 2053  803 2052 1954  979 1418 2051
  524  527 2053  649 2053  280 2053  524  521 2010  649 2044    6 2333
 1446    6 1432 2044 2333  569  643 1171 2153 2202  527  649 2046 2308
 1103 1335  816 1789 2046  527 1045 1077 2046  612 

...

2149
[1191 2047 1208 2202 1177 2202  740 2202 1302 2047 1789  973 2053  736
 1194 1397 2047   88 1132 1146 2047  976 1777 1792 1845 1991   72 1878
 1988 2249  637 1318 1769 1777 1523 2044 1845 2044 1706  804 1727 1453
 2046 1304 2047  809  450 2333 1132 2044 2047 1169 1703 1994 1454  673
 2149 1289 2044  527 2044 2009 2046 1878  842 1454 1438 2246 1076 2044
 1903 1357 2052  571 1543 1132  644 1318 1318 2047 1492 1302 2008 1291
  805 2045 1792 2045 1432 1148  333  952 1237 1635 2151  624  521 1585
 1802   71 2149]
1777
[2047 1208 2202 1177 2202  740 2202 1302 2047 1789  973 2053  736 1194
 1397 2047   88 1132 1146 2047  976 1777 1792 1845 1991   72 1878 1988
 2249  637 1318 1769 1777 1523 2044 1845 2044 1706  804 1727 1453 2046
 1304 2047  809  450 2333 1132 2044 2047 1169 1703 1994 1454  673 2149
 1289 2044  527 2044 2009 2046 1878  842 1454 1438 2246 1076 2044 1903
 1357 2052  571 1543 1132  644 1318 1318 2047 1492 1302 2008 1291  805
 2045 1792 2045 1432 1148  333  952 1237 1635 2151

571
[2047   88 1132 1146 2047  976 1777 1792 1845 1991   72 1878 1988 2249
  637 1318 1769 1777 1523 2044 1845 2044 1706  804 1727 1453 2046 1304
 2047  809  450 2333 1132 2044 2047 1169 1703 1994 1454  673 2149 1289
 2044  527 2044 2009 2046 1878  842 1454 1438 2246 1076 2044 1903 1357
 2052  571 1543 1132  644 1318 1318 2047 1492 1302 2008 1291  805 2045
 1792 2045 1432 1148  333  952 1237 1635 2151  624  521 1585 1802   71
 2149 1777 1568 1075 1454 1320  891 1168 1585  974 1529  644 1902 1705
 1147 1585  571]
1251
[  88 1132 1146 2047  976 1777 1792 1845 1991   72 1878 1988 2249  637
 1318 1769 1777 1523 2044 1845 2044 1706  804 1727 1453 2046 1304 2047
  809  450 2333 1132 2044 2047 1169 1703 1994 1454  673 2149 1289 2044
  527 2044 2009 2046 1878  842 1454 1438 2246 1076 2044 1903 1357 2052
  571 1543 1132  644 1318 1318 2047 1492 1302 2008 1291  805 2045 1792
 2045 1432 1148  333  952 1237 1635 2151  624  521 1585 1802   71 2149
 1777 1568 1075 1454 1320  891 1168 1585  974 1529 

2044
[1769 1777 1523 2044 1845 2044 1706  804 1727 1453 2046 1304 2047  809
  450 2333 1132 2044 2047 1169 1703 1994 1454  673 2149 1289 2044  527
 2044 2009 2046 1878  842 1454 1438 2246 1076 2044 1903 1357 2052  571
 1543 1132  644 1318 1318 2047 1492 1302 2008 1291  805 2045 1792 2045
 1432 1148  333  952 1237 1635 2151  624  521 1585 1802   71 2149 1777
 1568 1075 1454 1320  891 1168 1585  974 1529  644 1902 1705 1147 1585
  571 1251 1291 1635 1902 1777 1430  644 1727  569 1251 1292 1646   70
 1902  348 2044]
1777
[1777 1523 2044 1845 2044 1706  804 1727 1453 2046 1304 2047  809  450
 2333 1132 2044 2047 1169 1703 1994 1454  673 2149 1289 2044  527 2044
 2009 2046 1878  842 1454 1438 2246 1076 2044 1903 1357 2052  571 1543
 1132  644 1318 1318 2047 1492 1302 2008 1291  805 2045 1792 2045 1432
 1148  333  952 1237 1635 2151  624  521 1585 1802   71 2149 1777 1568
 1075 1454 1320  891 1168 1585  974 1529  644 1902 1705 1147 1585  571
 1251 1291 1635 1902 1777 1430  644 1727  569 1251

1893
[1132 2044 2047 1169 1703 1994 1454  673 2149 1289 2044  527 2044 2009
 2046 1878  842 1454 1438 2246 1076 2044 1903 1357 2052  571 1543 1132
  644 1318 1318 2047 1492 1302 2008 1291  805 2045 1792 2045 1432 1148
  333  952 1237 1635 2151  624  521 1585 1802   71 2149 1777 1568 1075
 1454 1320  891 1168 1585  974 1529  644 1902 1705 1147 1585  571 1251
 1291 1635 1902 1777 1430  644 1727  569 1251 1292 1646   70 1902  348
 2044 1777  569 2047 1074 2052 1292 2048 1412  613 2044  509  957 1845
 1635 2252 1893]
10
[2044 2047 1169 1703 1994 1454  673 2149 1289 2044  527 2044 2009 2046
 1878  842 1454 1438 2246 1076 2044 1903 1357 2052  571 1543 1132  644
 1318 1318 2047 1492 1302 2008 1291  805 2045 1792 2045 1432 1148  333
  952 1237 1635 2151  624  521 1585 1802   71 2149 1777 1568 1075 1454
 1320  891 1168 1585  974 1529  644 1902 1705 1147 1585  571 1251 1291
 1635 1902 1777 1430  644 1727  569 1251 1292 1646   70 1902  348 2044
 1777  569 2047 1074 2052 1292 2048 1412  613 2044  

2049
[ 842 1454 1438 2246 1076 2044 1903 1357 2052  571 1543 1132  644 1318
 1318 2047 1492 1302 2008 1291  805 2045 1792 2045 1432 1148  333  952
 1237 1635 2151  624  521 1585 1802   71 2149 1777 1568 1075 1454 1320
  891 1168 1585  974 1529  644 1902 1705 1147 1585  571 1251 1291 1635
 1902 1777 1430  644 1727  569 1251 1292 1646   70 1902  348 2044 1777
  569 2047 1074 2052 1292 2048 1412  613 2044  509  957 1845 1635 2252
 1893   10 2046 1194 1171 1948   10 1396 1358 1646 1954   10 1454 1292
 1310 1169 2049]
2009
[1454 1438 2246 1076 2044 1903 1357 2052  571 1543 1132  644 1318 1318
 2047 1492 1302 2008 1291  805 2045 1792 2045 1432 1148  333  952 1237
 1635 2151  624  521 1585 1802   71 2149 1777 1568 1075 1454 1320  891
 1168 1585  974 1529  644 1902 1705 1147 1585  571 1251 1291 1635 1902
 1777 1430  644 1727  569 1251 1292 1646   70 1902  348 2044 1777  569
 2047 1074 2052 1292 2048 1412  613 2044  509  957 1845 1635 2252 1893
   10 2046 1194 1171 1948   10 1396 1358 1646 1954

In [143]:
def midi_convert(predicted_notes):
    '''
    convert the notes in predicted_notes to midi files
    
    predicted_notes: Output from function predict_to_notes() --> list of predicted notes
    
    returns: None --- > Creates a midi file when ran
    '''
    offset = 0 
    midi_notes = []
    
    #create notes, chords, and rest objects from predicted_notes
    for pattern in predicted_notes:
        pattern = pattern.split(' ')
        print(pattern)
        temp = pattern[0]
        duration = pattern[1]
        pattern = temp
        #checking to see if a note is a chord 
        if ('.' in pattern) or pattern.isdigit():
            chord_notes = pattern.split('.')
            notes = []
            for current_note in chord_notes:
                this_note = note.Note(int(current_note))
                this_note.storedInstrument = instrument.Piano()
                notes.append(this_note)
            new_chord = chord.Chord(notes) 
            new_chord.offset = offset
            midi_notes.append(new_chord)
        #if the pattern is a rest    
        elif ('rest' in pattern):
            this_rest = note.Rest(pattern)
            this_rest.offset = offset
            this_rest.storedInstrument = instrument.Piano() #Still needs to be paino instrument even though = rest
            midi_notes.append(this_rest)
        else:
            this_note = note.Note(pattern)
            this_note.offset = offset 
            this_note.storedInstrument = instrument.Piano()
            midi_notes.append(this_note)
        #ensure that the notes do not stack    
        offset += convert_to_float(duration)
    
    midi = stream.Stream(midi_notes)
    midi.write('midi', fp = 'midi_output/big_batch_attention_no_time.mid')
    
    
#From: https://stackoverflow.com/questions/1806278/convert-fraction-to-float
def convert_to_float(frac_str):
    try:
        return float(frac_str)
    except ValueError:
        num, denom = frac_str.split('/')
        try:
            leading, num = num.split(' ')
            whole = float(leading)
        except ValueError:
            whole = 0
        frac = float(num) / float(denom)
        return whole - frac if whole < 0 else whole + frac

In [109]:
def generate():
    '''
    generates the midi file
    
    
    
    output:None ---> creates the midi file
    
    '''
    #Load the notes
    pickle_file = open("data/notes", "rb")
    notes = pickle.load(pickle_file)
    
    #create the pitchnames
    pitches = sorted(set(note for note in notes))
    
    
    lstm_input, lstm_normalized = prepare_notes(notes, pitches, VOCAB) 
    model = tf.keras.models.load_model('models_weights/time_dist_no_attention.h5')
    model.load_weights('models_weights/weights_no_attentnion.hdf5)
    predicted_notes = predict_to_notes(model, lstm_input, pitches, VOCAB)
    midi_convert(predicted_notes)
    

SyntaxError: EOL while scanning string literal (<ipython-input-109-77e52bb5364a>, line 20)

In [16]:
model.load_weights('models_weights/weights_no_attentnion.hdf5')


In [134]:
#model = tf.keras.models.load_model('models_weights/attention_model.h5')

In [144]:
#generate()
midi_convert(predicted_notes)

['G2', '4.0']
['E-4', '8.75']
['G2', '4.0']
['C#5', '1.75']
['C7', '5/3']
['G3', '1.0']
['G3', '1.75']
['C#5', '1.75']
['9.11.4', '2.0']
['G3', '1.0']
['G3', '1.75']
['G3', '1.75']
['0.4', '0.5']
['7.11.2', '8.0']
['G3', '1.75']
['6.10', '1.0']
['G3', '1.5']
['F4', '2/3']
['7.9.1', '1/3']
['C#5', '4.0']
['G3', '1.0']
['3.4.8', '3.0']
['3.5.7.10.0', '0.25']
['G3', '1.75']
['4.7.11', '2.5']
['G3', '1.75']
['11', '1.0']
['G3', '1.75']
['3.4.8', '3.0']
['3', '2/3']
['G#4', '4.0']
['4.7.11', '2.5']
['G2', '3.75']
['0', '1/3']
['rest', '5.25']
['C3', '2.0']
['0', '1/3']
['C2', '0.25']
['G2', '3.75']
['rest', '5.25']
['3.7.11', '0.25']
['4.7.11', '0.5']
['9.2', '4/3']
['rest', '134/3']
['rest', '193/6']
['3.5.7.10.0', '0.25']
['4.7.11', '2.5']
['G2', '4/3']
['rest', '40/3']
['9.0.4', '1.75']
['B3', '2.75']
['6.10.1', '2.0']
['E5', '0.5']
['G2', '4/3']
['3.5.7.10.0', '0.25']
['8.11.2', '1.0']
['9.0', '1.0']
['G2', '4/3']
['4.6', '1.0']
['A2', '2/3']
['G#2', '4/3']
['E4', '1/3']
['6.9.1', '2.5'

In [139]:
#model = tf.keras.models.load_model('models_weights/time_dist_no_attention.h5')
path = 'models_weights/bb_att_model_only.h5'

In [140]:
model = tf.keras.models.load_model((path),custom_objects={'SeqSelfAttention': SeqSelfAttention})
model.load_weights('models_weights/bb_att_only_weights.h5')

In [ ]:
midi_convert(predicted_notes)

In [141]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_37 (Bidirectio (None, 100, 1024)         2105344   
_________________________________________________________________
seq_self_attention_35 (SeqSe (None, None, 1024)        65601     
_________________________________________________________________
dropout_37 (Dropout)         (None, None, 1024)        0         
_________________________________________________________________
batch_normalization_13 (Batc (None, None, 1024)        4096      
_________________________________________________________________
bidirectional_38 (Bidirectio (None, None, 1024)        6295552   
_________________________________________________________________
seq_self_attention_36 (SeqSe (None, None, 1024)        65601     
_________________________________________________________________
dense_23 (Dense)             (None, None, 256)       

In [114]:
import keras
from keras_self_attention import SeqSelfAttention

In [115]:
model = tf.keras.models.load_model(path)


In [118]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_3 (Bidirection (None, 100, 1024)         2105344   
_________________________________________________________________
dropout_3 (Dropout)          (None, 100, 1024)         0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 100, 1024)         4096      
_________________________________________________________________
bidirectional_4 (Bidirection (None, 100, 1024)         6295552   
_________________________________________________________________
dropout_4 (Dropout)          (None, 100, 1024)         0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 100, 1024)         4096      
_________________________________________________________________
bidirectional_5 (Bidirection (None, 1024)             

In [74]:
note_dict = dict((number, note) for number, note in enumerate(pitches))

In [75]:
note_dict 

{0: '0.1.3.7.81.0',
 1: '0.1.5.80.25',
 2: '0.1.5.80.5',
 3: '0.1.5.81.0',
 4: '0.1.50.75',
 5: '0.1.60.5',
 6: '0.1.62.5',
 7: '0.10.0',
 8: '0.2.3.5.70.5',
 9: '0.2.4.70.5',
 10: '0.2.4.71.0',
 11: '0.2.40.5',
 12: '0.2.5.70.5',
 13: '0.2.50.5',
 14: '0.2.52/3',
 15: '0.2.60.25',
 16: '0.2.60.5',
 17: '0.2.61/3',
 18: '0.2.62/3',
 19: '0.2.64/3',
 20: '0.2.70.5',
 21: '0.2.70.75',
 22: '0.2.71.0',
 23: '0.2.71/3',
 24: '0.20.0',
 25: '0.20.25',
 26: '0.20.5',
 27: '0.20.75',
 28: '0.21.0',
 29: '0.21/3',
 30: '0.22/3',
 31: '0.3.50.5',
 32: '0.3.51/3',
 33: '0.3.52/3',
 34: '0.3.6.91/3',
 35: '0.3.6.94.0',
 36: '0.3.60.5',
 37: '0.3.62/3',
 38: '0.3.70.25',
 39: '0.3.70.5',
 40: '0.3.70.75',
 41: '0.3.71.0',
 42: '0.3.71.5',
 43: '0.3.71/3',
 44: '0.30.25',
 45: '0.30.5',
 46: '0.30.75',
 47: '0.31.0',
 48: '0.31.5',
 49: '0.31.75',
 50: '0.31/3',
 51: '0.33.0',
 52: '0.34/3',
 53: '0.4.52.0',
 54: '0.4.60.5',
 55: '0.4.60.75',
 56: '0.4.70.25',
 57: '0.4.70.5',
 58: '0.4.70.75',
 59